# Assingment 2 and 5-- By Aryan Rathore

## this files gets the BIO labels of any number of files for assingment 2 and 5

## 1. open the files in the text folder
## 2. generate the BIO/IOB labels for a given text
## 3. convert the BIO/IOB labels into format present in the original folder
## 4. saving the results in original2 folder
## 5. using the gemini-1.5-flash-latest to get the correct labels
## 6. parsing the gemini reponse and saving the results original2 folder

### Note: due to hardware limitation i couldnt use the models that are the best for the given task. i used the models from 1-6 which does recognize the need words most of the time but fails in successfully classifing them. hence i used the pre loaded gemini model through which i was able to get the correct labels

models experimented with:
1. d4data/biomedical-ner-all
2. cross-encoder/nli-distilroberta-base model
3. ugaray96/biobert_ncbi_disease_ner
4. ugaray96/biobert_ncbi_disease_ner
5. dslim/bert-base-NER
6. samrawal/bert-base-uncased_clinical-ner

## --- 1. open the files in the text folder ---

In [1]:
with open('/content/drive/MyDrive/cadec/text/ARTHROTEC.1.txt', 'r') as e:
  text1 = e.read()
with open('/content/drive/MyDrive/cadec/text/ARTHROTEC.2.txt', 'r') as f:
  text2 = f.read()
with open('/content/drive/MyDrive/cadec/text/ARTHROTEC.3.txt', 'r') as g:
  text3 = g.read()
with open('/content/drive/MyDrive/cadec/text/ARTHROTEC.4.txt', 'r') as h:
  text4 = h.read()
with open('/content/drive/MyDrive/cadec/text/ARTHROTEC.5.txt', 'r') as i:
  text5 = i.read()

In [ ]:
'''
    the following code may or may not reduce accuracy
'''

import spacy
import re

# !python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")

def distill_text(text):
    """
    input: text : output: list of words
    Aggressively distills text down to its most meaningful words. saves computation and most needed words are labeled

    """

    doc = nlp(text)

    # Define an EXTENSIVE custom stop list.
    extended_stop_words = {
        "good", "great", "nice", "really", "very", "quite", "pretty", "just", "maybe", "well",
        "bit", "little", "actually", "basically", "literally", "seriously", "probably",
        "perhaps", "something", "thing", "anything", "everything", "someone", "anyone",
        "get", "go", "see", "know", "think", "like", "look", "want", "need", "use", "take", "make",
        "say", "tell", "come", "give", "also", "still", "even", "since", "however", "therefore"
    }

    filtered_tokens = []
    for token in doc:
        # Filter 1: Remove standard stop words
        if token.is_stop:
            continue

        # Filter 2: Remove punctuation, spaces, numbers
        if token.is_punct or token.is_space or token.like_num:
            continue

        # Filter 3: Remove tokens with non-alphabetical characters (keeps hyphenated words)
        if not re.match(r'^[a-zA-Z-]+$', token.text):
            continue

        # Filter 4: Remove short words (length < 2)
        if len(token.text) < 3:
            continue

        # Filter 5: Remove verbs that are too common (optional, can be aggressive)
        if token.pos_ == "VERB" and token.lemma_ in {"be", "have", "do", "say", "get", "make", "go", "know"}:
            continue

        # Filter 6: Remove custom stop words
        if token.lemma_.lower() in extended_stop_words:
            continue

        # Filter 7: Keep only Nouns, Adjectives, and meaningful Verbs
        if token.pos_ in ["NOUN", "ADJ", "VERB", "ADV"]: # PROPN for proper nouns
            filtered_tokens.append(token.lemma_.lower())

    return filtered_tokens

# ------- Example Usage -------

text1 = " ".join(distill_text(text1))
text2 = " ".join(distill_text(text2))
text3 = " ".join(distill_text(text3))
text4 = " ".join(distill_text(text4))
text5 = " ".join(distill_text(text5))

text1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 28.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


'feel drowsy blurred vision far gastric problem arthrotec year arthritis progressively bad point tear agony start day day month far pain feel weird'

# --- 2. generate the BIO/IOB labels for a given text ---

In [ ]:
import os
from transformers import pipeline

# Use a smaller, efficient model that's available
classifier = pipeline("zero-shot-classification",
                      model="ugaray96/biobert_ncbi_disease_ner")


labels = ["ADR", "Disease", "Symptom", "Outside"]

def label_text_with_bio(text):
    # Split text into words
    words = text.split()
    bio_labels = []

    # Track current entity type for I-tags
    current_entity = None

    for word in words:
        # Simple heuristic to reduce API calls for obvious cases
        if len(word) <= 2 or word.lower() in ['the', 'and', 'is', 'are', 'of', 'a', 'an', 'in', 'on', 'at']:
            # If we're in an entity, mark as Inside, otherwise Outside
            if current_entity:
                bio_labels.append(f"I-{current_entity}")
            else:
                bio_labels.append("O")
            continue

        try:
            result = classifier(word, candidate_labels=labels)
            predicted_label = result['labels'][0]

            if predicted_label == "Outside":
                # End of any current entity
                bio_labels.append("O")
                current_entity = None
            else:
                # If same entity as before, mark as Inside
                if current_entity == predicted_label:
                    bio_labels.append(f"I-{predicted_label}")
                else:
                    # New entity starts
                    bio_labels.append(f"B-{predicted_label}")
                    current_entity = predicted_label
        except:
            # Fallback if model fails - continue current entity or mark as Outside
            if current_entity:
                bio_labels.append(f"I-{current_entity}")
            else:
                bio_labels.append("O")

    return words, bio_labels


# Label the text
words1, bio_labels1 = label_text_with_bio(text1)
print("Text1 completed")
words2, bio_labels2 = label_text_with_bio(text2)
print("Text2 completed")
words3, bio_labels3 = label_text_with_bio(text3)
print("Text3 completed")
words4, bio_labels4 = label_text_with_bio(text4)
print("Text4 completed")
words5, bio_labels5 = label_text_with_bio(text5)
print("Text5 completed")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ugaray96/biobert_ncbi_disease_ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Text1 completed
Text2 completed
Text3 completed
Text4 completed
Text5 completed


In [ ]:
print("--------------- BIO labels for text1: ---------------------")

o,i,b =0,0,0
for word, label in zip(words1, bio_labels1):
  if str(label[0]) == "O":
    o += 1
    # continue # ignore the words labeled "O"
  elif str(label[0]) == "B":
    b += 1
  elif str(label[0]) == "I":
    i += 1
  print(f"{word}: {label}")
print(f"\nlabel O count: {o}")
print(f"label I count: {i}")
print(f"label B count: {b}")

--------------- BIO labels for text1: ---------------------
feel: O
drowsy: O
blurred: B-ADR
vision: O
far: O
gastric: O
problem: B-ADR
arthrotec: I-ADR
year: I-ADR
arthritis: O
progressively: B-ADR
bad: I-ADR
point: I-ADR
tear: I-ADR
agony: I-ADR
start: I-ADR
day: B-Disease
day: I-Disease
month: I-Disease
far: O
pain: O
feel: O
weird: B-ADR

label O count: 9
label I count: 9
label B count: 5


## --- 3. convert the BIO/IOB labels into format present in the original folder ---

In [ ]:
# Function to convert BIO to original format
def bio_to_original_format(words, bio_labels):
    entities = []
    current_entity = None
    start_idx = 0

    for i, (word, label) in enumerate(zip(words, bio_labels)):
        if label.startswith("B-"):
            # If we were tracking an entity, save it
            if current_entity:
                entity_text = " ".join(words[start_idx:i])
                entities.append({
                    "tag": f"T{len(entities)+1}",
                    "label": current_entity,
                    "text": entity_text
                })

            # Start new entity
            current_entity = label[2:]  # Remove "B-" prefix
            start_idx = i

        elif label == "O" and current_entity:
            # End of entity
            entity_text = " ".join(words[start_idx:i])
            entities.append({
                "tag": f"T{len(entities)+1}",
                "label": current_entity,
                "text": entity_text
            })
            current_entity = None

    # Handle the last entity if exists
    if current_entity:
        entity_text = " ".join(words[start_idx:])
        entities.append({
            "tag": f"T{len(entities)+1}",
            "label": current_entity,
            "text": entity_text
        })

    return entities
result1 = bio_to_original_format(words1, bio_labels1)
result2 = bio_to_original_format(words2, bio_labels2)
result3 = bio_to_original_format(words3, bio_labels3)
result4 = bio_to_original_format(words4, bio_labels4)
result5 = bio_to_original_format(words5, bio_labels5)

In [ ]:
result1

[{'tag': 'T1', 'label': 'ADR', 'text': 'blurred'},
 {'tag': 'T2', 'label': 'ADR', 'text': 'problem arthrotec year'},
 {'tag': 'T3',
  'label': 'ADR',
  'text': 'progressively bad point tear agony start'},
 {'tag': 'T4', 'label': 'Disease', 'text': 'day day month'},
 {'tag': 'T5', 'label': 'ADR', 'text': 'weird'}]

In [ ]:
import pandas as pd

result1 = pd.DataFrame(result1)
result2 = pd.DataFrame(result2)
result3 = pd.DataFrame(result3)
result4 = pd.DataFrame(result4)
result5 = pd.DataFrame(result5)


In [ ]:
result3

,tag,label,text
0,T1,Symptom,nausea
1,T2,Disease,frequent
2,T3,Symptom,hunger
3,T4,ADR,sharp
4,T5,Disease,cramping
5,T6,Disease,low
6,T7,ADR,gut lot difference
7,T8,ADR,suppose treat
8,T9,Disease,day
9,T10,ADR,prefer


# --- 4. saving the results in original2 folder ---

In [ ]:
result1.to_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.1.txt")
result2.to_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.2.txt")
result3.to_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.3.txt")
result4.to_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.4.txt")
result5.to_csv("/content/drive/MyDrive/cadec/original2/ARTHROTEC.5.txt")

## ---  5. using the gemini-1.5-flash-latest to get the correct labels ---

### since the hugging face models were not working correctly i am using the preloaded gemini model to get the correct labels

In [3]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GEMINI_KEY'))

In [ ]:
import json

# the code takes input for 50 files and saves the model results

# texts = []

# for i in range(1, 51):
#   with open(f'/content/drive/MyDrive/cadec/text/ARTHROTEC.{i}.txt', 'r') as a:
#     content = a.read()
#     texts.append(content)

model = genai.GenerativeModel('gemini-1.5-flash-latest')

'''
    the code goes makes a diffrent prompt for each different text and saves the output in the gemini results list

'''

# the code takes input for 50 files and saves the model results

texts = []

for i in range(1, 51):
  with open(f'/content/drive/MyDrive/cadec/text/ARTHROTEC.{i}.txt', 'r') as a:
    content = a.read()
    texts.append(content)

gemini_results = []

for text in texts:
    ner_prompt = f"""
Extract Named Entities from the following text based on the provided BIO beggining inside outside labels.

Labels: 'Adeverse Drug Reaction (ADR)', 'Disease', 'Symptom', 'Drug'.

For each extracted entity, provide the exact text from the input and its corresponding label.

a typical file for example : I feel a bit drowsy & have a little blurred vision, so far no gastric problems.
I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.
Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it.
every day for the next month to see how I get on, here goes.
So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.

its correct example output labels:
ADR | 9 19  | bit drowsy
ADR | 29 50 | little blurred vision
Drug | 93 102 | Arthrotec
Disease | 179 188 | arthritis
Symptom | 260 265 | agony
ADR | 62 78 | gastric problems
Symptom | 412 417 | pains
ADR | 437 453 | feel a bit weird

Return the results as text with similar output as the example output.
If no entities are found, return an empty text.

find labels for Text given below:

{text}

"""

    response = model.generate_content(ner_prompt)


    try:
        entities = response.text
        gemini_results.append(entities)

    except json.JSONDecodeError: # if some error occurs

        print(f"Warning: Could not parse JSON from response for text: {text[:50]}...")
        gemini_results.append([])

# saving results
# for 50 files
# gemini_results

gemini_results1, gemini_results2, gemini_results3, gemini_results4, gemini_results5 = gemini_results

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15
Please retry in 30.568034036s.



## ---- 6. parsing the gemini reponse and saving the results original2 folder ----

In [ ]:
# for 50 files
# new_result = []
# for result in gemini_results:
#   result = result.split("\n")[:-2]
#   new_result.append(result)

gemini_result1 = gemini_result1.split("\n")[:-2]
gemini_result2 = gemini_result2.split("\n")[:-2]
gemini_result3 = gemini_result3.split("\n")[:-2]
gemini_result4 = gemini_result4.split("\n")[:-2]
gemini_result5 = gemini_result5.split("\n")[:-2]

In [50]:
gemini_result1

['ADR | 9 19 | bit drowsy',
 'ADR | 29 50 | little blurred vision',
 'Drug | 93 102 | Arthrotec',
 'Disease | 179 188 | arthritis',
 'Symptom | 260 265 | agony',
 'ADR | 62 78 | gastric problems',
 'Symptom | 412 417 | pains',
 'ADR | 437 453 | feel a bit weird']

In [ ]:
import pandas as pd
import os

output_dir = "/content/drive/MyDrive/cadec/original2/"
os.makedirs(output_dir, exist_ok=True) # Create directory if it doesn't exist


def parse_gemini_output(gemini_output_list, file_name):
    """
    Parses the Gemini output strings into a list of dictionaries by splitting each line by '|'.
    Each dictionary represents an entity with 'file_name', 'label', and 'text'.
    """

    entities = []
    for i, line in enumerate(gemini_output_list):
        parts = line.split('|')
        label = parts[0].strip()
        text = parts[2].strip()
        entities.append({"identifier": label, "text": text, "file_name": file_name})

    return entities

# parse gemini results for 50 files
# for i, result in enumerate(new_result):
#   parsed_result = parse_gemini_output(result, f"ARTHROTEC.{1}.txt")
#   converted_result = pd.DataFrame(parsed_result)
#   converted_result.to_csv(os.path.join(output_dir, "ARTHROTEC.{i}.txt"), index=False)

# Parse each Gemini result with the corresponding file name
parsed_result1 = parse_gemini_output(gemini_result1, "ARTHROTEC.1.txt")
parsed_result2 = parse_gemini_output(gemini_result2, "ARTHROTEC.2.txt")
parsed_result3 = parse_gemini_output(gemini_result3, "ARTHROTEC.3.txt")
parsed_result4 = parse_gemini_output(gemini_result4, "ARTHROTEC.4.txt")
parsed_result5 = parse_gemini_output(gemini_result5, "ARTHROTEC.5.txt")

# Convert parsed results to DataFrames
converted_result1 = pd.DataFrame(parsed_result1)
converted_result2 = pd.DataFrame(parsed_result2)
converted_result3 = pd.DataFrame(parsed_result3)
converted_result4 = pd.DataFrame(parsed_result4)
converted_result5 = pd.DataFrame(parsed_result5)

# Display the first converted result
converted_result1

,identifier,text,file_name
0,ADR,bit drowsy,ARTHROTEC.1.txt
1,ADR,little blurred vision,ARTHROTEC.1.txt
2,Drug,Arthrotec,ARTHROTEC.1.txt
3,Disease,arthritis,ARTHROTEC.1.txt
4,Symptom,agony,ARTHROTEC.1.txt
5,ADR,gastric problems,ARTHROTEC.1.txt
6,Symptom,pains,ARTHROTEC.1.txt
7,ADR,feel a bit weird,ARTHROTEC.1.txt


In [56]:
import os

output_dir = "/content/drive/MyDrive/cadec/original2/"
os.makedirs(output_dir, exist_ok=True) # Create directory if it doesn't exist

converted_result1.to_csv(os.path.join(output_dir, "ARTHROTEC.1.txt"), index=False)
converted_result2.to_csv(os.path.join(output_dir, "ARTHROTEC.2.txt"), index=False)
converted_result3.to_csv(os.path.join(output_dir, "ARTHROTEC.3.txt"), index=False)
converted_result4.to_csv(os.path.join(output_dir, "ARTHROTEC.4.txt"), index=False)
converted_result5.to_csv(os.path.join(output_dir, "ARTHROTEC.5.txt"), index=False)

print("Results saved successfully!")

Results saved successfully!
